In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [3]:
from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [4]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

2021-02-16 18:13:52,402 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-16 18:13:52,403 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2021-02-16 18:13:52,404 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2021-02-16 18:13:52,408 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [5]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [6]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [7]:
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

2021-02-16 18:13:56,270 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2021-02-16 18:13:56,274 : INFO : saving sparse matrix to /tmp/deerwester.mm
2021-02-16 18:13:56,275 : INFO : PROGRESS: saving document #0
2021-02-16 18:13:56,279 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2021-02-16 18:13:56,281 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [8]:
from smart_open import open

In [9]:
class MyCorpus:
    def __iter__(self):
        for line in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [10]:
corpus_memory_friendly = MyCorpus()

In [11]:
print(corpus_memory_friendly)

In [12]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [13]:
dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))

2021-02-16 18:20:04,179 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-16 18:20:04,182 : INFO : built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)


In [14]:
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [15]:
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

In [16]:
corpora.MmCorpus.serialize('/tmp/corpus.mm', corpus)

2021-02-16 18:22:39,295 : INFO : storing corpus in Matrix Market format to /tmp/corpus.mm
2021-02-16 18:22:39,298 : INFO : saving sparse matrix to /tmp/corpus.mm
2021-02-16 18:22:39,300 : INFO : PROGRESS: saving document #0
2021-02-16 18:22:39,303 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2021-02-16 18:22:39,305 : INFO : saving MmCorpus index to /tmp/corpus.mm.index


In [17]:
corpus = corpora.MmCorpus('/tmp/corpus.mm')

2021-02-16 18:24:18,522 : INFO : loaded corpus index from /tmp/corpus.mm.index
2021-02-16 18:24:18,526 : INFO : initializing cython corpus reader from /tmp/corpus.mm
2021-02-16 18:24:18,530 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


In [18]:
print(corpus)

MmCorpus(2 documents, 2 features, 1 non-zero entries)
